In [14]:
from ioh import get_problem, ProblemClass
from ioh import logger
import sys
import numpy as np
import time

In [15]:
# Declaration of problems to be tested.
# We obtain an interface of the OneMax problem here.
dimension = 50

In [16]:
"""
1 (fid) : The funciton ID of the problem in the problem suite. OneMax is 1 defined within the PBO class. 2 would correspond to another problem.
dimension : The dimension of the problem, which we have set to 50.
instance: In benchmarking libraries, problems often have multiple instances. These instances may vary slightly (e.g., different random noise, shifts, etc.) 
            to allow algorithms to be tested on a variety of conditions.
om(x) return the fitness value of 'x'
"""
om = get_problem(1, dimension=dimension, instance=1, problem_class=ProblemClass.PBO)
# We know the optimum of onemax
optimum = dimension

In [17]:
# Create default logger compatible with IOHanalyzer
# `root` indicates where the output files are stored.
# `folder_name` is the name of the folder containing all output. You should compress the folder 'run' and upload it to IOHanalyzer.
l = logger.Analyzer(root="data", 
    folder_name="run", 
    algorithm_name="genetic_algorithm", 
    algorithm_info="The lab session of the evolutionary algorithm course in LIACS")

In [18]:
om.attach_logger(l)

In [19]:
# Parameters setting
pop_size = 10
tournament_k = 5
mutation_rate = 0.02
crossover_probability = 0.5

In [20]:
# Uniform Crossover
def crossover(p1, p2):
    if np.random.rand() < crossover_probability:
        cross_points = np.random.randint(0, 2, size=len(p1)).astype(np.bool_)
        p1[cross_points] = p2[cross_points]
    return p1

In [21]:
# Standard bit mutation using mutation rate p
def mutation(p):
    for mutation_point in range(len(p)):
        if np.random.rand() < mutation_rate:
            p[mutation_point] = 1 if p[mutation_point] == 0 else 0
    return p


In [22]:
# Using the Fitness proportional selection
def mating_seletion(parent, parent_f) :
    parent = np.array(parent)
    parent_f = np.array(parent_f)  
    fitness_sum = np.sum(parent_f)
    fitness = parent_f / fitness_sum
    fitness_sort = np.argsort(fitness)[::-1]
    return parent[fitness_sort][:pop_size]

In [23]:
def genetic_algorithm(func, budget = None):
    
    # budget of each run: 10000
    if budget is None:
        budget = 10000
    
    # f_opt : Optimal function value.
    # x_opt : Optimal solution.
    f_opt = sys.float_info.min
    x_opt = None
    
    # parent : A list that holds the binary strings representing potential solutions or individuals in the current population.
    # parent_f : A list that holds the fitness values corresponding to each individual in the parent list.
    parent = []
    parent_f = []
    for i in range(pop_size):
        # Initialization
        parent.append(np.random.randint(2, size = func.meta_data.n_variables))
        parent_f.append(func(parent[i]))
        budget = budget - 1
    
    while (f_opt < optimum and budget > 0):
        parent = np.array(parent)
        parent_original = np.copy(parent)
        parent_f = np.array(parent_f)
        parent_f_original = np.copy(parent_f)
        # Perform mating selection, crossover, and mutation to generate offspring
        '''parent_index =  np.random.choice(len(parent), size = tournament_k, replace=False)
        parent = parent[parent_index]
        parent_f = parent_f[parent_index]

        fitness_sum = np.sum(parent_f)
        fitness = parent_f / fitness_sum'''
        fitness_sort = np.argsort(parent_f)[::-1][:2]
        parent = parent[fitness_sort]
        offspring = mutation(crossover(parent[0], parent[1]))
        
        parent = list(parent_original)
        parent.append(offspring)
        parent_f = list(parent_f_original)
        parent_f.append(func(offspring))
        budget = budget - 1
        
        '''fitness_sort = np.argsort(fitness)[:2]
        parent = list(parent[fitness_sort])
        parent_f = list(parent_f[fitness_sort])
        

        for i in range(len(parent)):
            for j in range(i+1, len(parent)):
                offspring = mutation(crossover(parent[i], parent[j]))
                parent.append(offspring)
                parent_f.append(func(offspring))
                budget = budget - 1'''

        parent = mating_seletion(parent, parent_f)
        parent_f = []
        for i in range(pop_size):
            parent_f.append(func(parent[i]))
        if func(parent[0])>f_opt:
            f_opt = func(parent[0])
            x_opt = parent[0]
    # ioh function, to reset the recording status of the function.
    func.reset()
    print(f_opt,x_opt)
    return f_opt, x_opt

In [24]:
def main():
    # We run the algorithm 20 independent times.
    for _ in range(20):
        genetic_algorithm(om)

In [25]:
if __name__ == '__main__':
  start = time.time()
  main()
  end = time.time()
  print("The program takes %s seconds" % (end-start))

50.0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
50.0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
50.0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
50.0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
50.0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
50.0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
50.0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
50.0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
50.0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
50.0 [1 1 1 1 1 1 1 1 1 1 1 